This Colab Notebook is for the Capstone Project implemented as a part of the IISc Deep Learning Course. The project topic is "Prompt Learning by Vision Transformer (VIT)"

We shall be using the prior work done in here:
https://github.com/KaiyangZhou/Dassl.pytorch

**!! WARNING !!** This notebook should be run only once for setting up the code / environment / downloading datasets

In [ ]:
# Initialize the project repo URL(s)
project_root="/content/drive/MyDrive/Work/IISc-DL/Capstone-Project"
dassl_project_repo_url="https://github.com/KaiyangZhou/Dassl.pytorch"
coop_project_repo_url="https://github.com/lomagoel/CoOp"

In [ ]:
import os

In [ ]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change to project directory and persist all files in there
%cd {project_root}

/content/drive/MyDrive/Work/IISc-DL/Capstone-Project


In [ ]:
# Clone 'dassl' project
# Check if the codebase is already downloaded
local_dassl_project_repo_dir = os.path.join(project_root, "Dassl.pytorch")
if(os.path.exists(local_dassl_project_repo_dir)):
  print("Dassl.pytorch repo already downloaded")
else:
  print("Downloading Dassl.pytorch repo ...")
  !git clone {dassl_project_repo_url}
  print("Done")

Dassl.pytorch repo already downloaded


In [ ]:
# Change to dassl project directory
%cd {local_dassl_project_repo_dir}
!pip install -r requirements.txt

In [ ]:
# Install this library (no need to re-build if the source code is modified)
!python setup.py develop

In [ ]:
# Clone 'CoOp' project inside 'dassl' project
# Check if the codebase is already downloaded
local_coop_project_repo_dir = os.path.join(local_dassl_project_repo_dir, "CoOp")
if(os.path.exists(local_coop_project_repo_dir)):
  print("CoOp repo already downloaded")
else:
  print("Downloading CoOp repo ...")
  !git clone {coop_project_repo_url}
  print("Done")

CoOp repo already downloaded


We will be performing the first experiment with **Caltech-101 Dataset**.
For this example, we are downloading the dataset from Kaggle.

In [ ]:
from google.colab import files

In [ ]:
# Complete the regular drill of Creating a Token in Kaggle and download the token
# in a JSON file. Now upload the JSON file from desktop to the Google cloud environment,
# so that the Google Colab notebook can use the JSON.

# Create a folder for the JSON file
!mkdir ~/.kaggle

# Upload the JSON file
files.upload()

# Move the JSON file and change permissions
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# CoOp Datasets folder
coop_project_dataset_dir = os.path.join(local_coop_project_repo_dir, "datasets")
# caltech-101 dataset directory
caltech_101_dataset_dir = os.path.join(coop_project_dataset_dir, "caltech-101")

In [ ]:
# Download caltech-101 dataset from kaggle
!kaggle datasets download imbikramsaha/caltech-101 --force -p {caltech_101_dataset_dir}

Dataset URL: https://www.kaggle.com/datasets/imbikramsaha/caltech-101
License(s): CC0-1.0
 81% 106M/131M [00:00<00:00, 282MB/s] 
100% 131M/131M [00:00<00:00, 268MB/s]


In [ ]:
# Extract images from the zip to a specific folder
caltech_101_dataset_zip_path = os.path.join(caltech_101_dataset_dir, "caltech-101.zip")

dest_folder = os.path.join(caltech_101_dataset_dir, "101_ObjectCategories")
!mkdir {dest_folder}
!unzip -q {caltech_101_dataset_zip_path} -d {dest_folder}

In [ ]:
# The images folder has to be massaged. The model expects the image file in the following
# structure. Image files should be in <dataset-dir>/caltech-101/101_ObjectCategories/<class>
# Currently the images are extracted under <dataset-dir>/caltech-101/101_ObjectCategories/caltech-101/<class>
# the followinc code will move the image class folder appropriately.

# Current folder is <dataset-dir>/caltech-101
# Target folder is <dataset-dir>/caltech-101/101_ObjectCategories/<class>

dirs = os.listdir(dest_folder)
for dir in dirs:
  if(dir == "BACKGROUND_Google"):
    continue
  dir_path = os.path.join(dest_folder, dir)
  image_folders = os.listdir(dir_path)
  for image_folder in image_folders:
    # move these image folders one level up
    image_folder_path = os.path.join(dir_path, image_folder)
    !mv {image_folder_path} {dest_folder}

In [ ]:
# Change to caltech-101 dataset folder
%cd {caltech_101_dataset_dir}

/content/drive/MyDrive/Work/IISc-DL/Capstone-Project/Dassl.pytorch/CoOp/datasets/caltech-101


In [ ]:
# Another JSON file is required to separate out the training and validation data
# JSON file is available in the following location:
# https://drive.google.com/file/d/1hyarUivQE36mY6jSomru6Fjd-JzwcCzN

# Suggest that the JSON is downloaded on to desktop and uploaded.
files.upload()


In [ ]:
# Move back to the root directory of "CoOp" project
%cd {local_coop_project_repo_dir}

/content/drive/MyDrive/Work/IISc-DL/Capstone-Project/Dassl.pytorch/CoOp


In [ ]:
# !!! WARNING !!!
# Optional step: if you want a clean output folder, uncomment the code line
# Suggest not to run it because data from all older runs can get removed

# !rm -rf output

# Now we are done with the code set-up and datasets

In [ ]:
# Execute the script for caltech-101 data
# CLIP + CoOp (M=16, end):
# 16 shots: bash scripts/coop/main.sh caltech101 rn50_ep50 end 16 6 False

!bash scripts/coop/main.sh caltech101 rn50_ep50 end 16 16 False

In [ ]:
# Results
!python parse_test_res.py output/caltech101/CoOp/rn50_ep50_16shots/nctx16_cscFalse_ctpend

Parsing files in output/caltech101/CoOp/rn50_ep50_16shots/nctx16_cscFalse_ctpend
file: output/caltech101/CoOp/rn50_ep50_16shots/nctx16_cscFalse_ctpend/seed1/log.txt. accuracy: 91.70%. 
file: output/caltech101/CoOp/rn50_ep50_16shots/nctx16_cscFalse_ctpend/seed2/log.txt. accuracy: 92.00%. 
file: output/caltech101/CoOp/rn50_ep50_16shots/nctx16_cscFalse_ctpend/seed3/log.txt. accuracy: 91.50%. 
===
Summary of directory: output/caltech101/CoOp/rn50_ep50_16shots/nctx16_cscFalse_ctpend
* accuracy: 91.73% +- 0.21%
===
